In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import os.path as osp
import sys
import glob
import time
import timeit
import math
import random

import itertools
from collections import OrderedDict

import re
from lxml import etree
from tqdm import tqdm, tnrange, tqdm_notebook

import numpy as np
import pandas as pd
import cv2
import imgaug as ia
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch import Tensor
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split, Subset

from draw import show_img, show_img_grid
from utils import postprocessing, letterbox_transforms, letterbox_label_reverse, fill_label_np_tensor, get_image_id_from_path
from boundingbox import CoordinateType, FormatType, BoundingBoxConverter
from transforms import IaaAugmentations, IaaLetterbox, ToTensor, Compose, \
                       iaa_hsv_aug, iaa_random_crop, iaa_letterbox, letterbox_reverse
from dataset import worker_init_fn, variable_shape_collate_fn
from darknet import YoloNet
# from evaluate import *


In [2]:
import json
from contextlib import contextmanager
from collections import OrderedDict
import pycocotools
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [3]:
seed = 0

def set_seed(seed):
    cudnn.benchmark = False
    cudnn.deterministic = True

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(seed)

In [4]:
cv2.setNumThreads(0)

In [5]:
weight_path = './yolov3.weights'
train_target_txt = "./data/coco/trainvalno5k.txt"
valid_target_txt = "./data/coco/5k.txt"

In [6]:
def load_coco(path):
    with open(path) as f:
        return [line.rstrip("\n") for line in f.readlines()]
    
coco_path = './data/coco.names'
classes = load_coco(coco_path)

# Pycocotools

## Sample

### Create Annotation File

In [7]:
def create_annotations(cat_list, img_list, ann_list):
    return OrderedDict({'categories': cat_list,
                        'images': img_list,
                        'annotations': ann_list})

def create_images_entry(image_id, width=None, height=None):
    if width is None or height is None:
        return OrderedDict({'id':image_id })
    else:
        return OrderedDict({'id':image_id, 'width':width, 'height':height })

def create_categories(class_names):
    return [{'id':i, 'name':cls} for i, cls in enumerate(class_names)]

def create_annotations_entry(image_id, bbox, category_id, ann_id, iscrowd=0, area=None, segmentation=None):
    if area is None:
        if segmentation is None:
            #Calulate area with bbox
            area = bbox[2] * bbox[3]
        else:
            raise NotImplementedError()
            
    return OrderedDict({
            "id": ann_id,
            "image_id": image_id,
            "category_id": category_id,
            "iscrowd": iscrowd,
            "area": area,
            "bbox": bbox
           })

In [8]:
img1 = create_images_entry(558840, 640, 427)
img_list = [img1]

In [9]:
a1 = create_annotations_entry(558840, [199.84, 200.46, 77.71, 70.88], 58, 156)       
a2 = create_annotations_entry(558840, [325.27, 104.38, 33.67, 105.99], 44, 370268)
a3 = create_annotations_entry(558840, [1.92, 87.91, 34.95, 175.35], 47, 676791) 

In [10]:
cat_list = create_categories(classes)

In [11]:
anno = [a1,a2, a3]

In [12]:
anno_json = create_annotations(cat_list, img_list, anno)

In [13]:
save_path = "coco_annotations.json"
with open(save_path, 'w') as f:
    json.dump(anno_json, f, indent=4, separators=(',', ':'))

### Create Results File

In [14]:
def create_results_entry(image_id, category_id, bbox, score):
    return OrderedDict({"image_id":image_id,
                        "category_id":category_id,
                        "bbox":bbox,
                        "score":score})

In [15]:
t1 = create_results_entry(558840, 58, [199.84, 200.46, 77.71, 70.88],  0.7)       
t2 = create_results_entry(558840, 44, [10.27, 104.38, 33.67, 105.99],  0.4)  #Bad bbox
t3 = create_results_entry(558840, 47, [1.92, 87.91, 34.95, 175.35],  0.65) 

In [16]:
coco_results = [t1,t2,t3]

In [17]:
save_path = "coco_results.json"
with open(save_path, 'w') as f:
    json.dump(coco_results, f, indent=4, separators=(',', ':'))

### Test the sample files

In [18]:
cocoGt = COCO("coco_annotations.json")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [19]:
resFile = './coco_results.json'
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!


In [20]:
imgIds=sorted(cocoDt.getImgIds())
imgIds

[558840]

In [21]:
cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.667
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.667
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.667
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.667
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=

## COCO Validation 5k Dataset

### Generate Ground Truth Annotation File

In [22]:
def get_img_ann_list(img_path_list, label_path_list):
    img_list, ann_list = [],[]
    for img_path, label_path in tqdm(zip(img_path_list, label_path_list), file=sys.stdout, leave=True, total=len(img_path_list)):
        image_id = get_image_id_from_path(img_path)
        # Read Image
        if osp.exists(img_path):
            img = cv2.imread(img_path)
        
        height, width = img.shape[0], img.shape[1]
        img_list.append(create_images_entry(image_id, width, height))
        # Read Labels
        if osp.exists(label_path):
            labels = np.loadtxt(label_path).reshape(-1,5)
        labels[..., 1:5] = BoundingBoxConverter.convert(labels[..., 1:5],
                                                     CoordinateType.Relative, FormatType.cxcywh,
                                                     CoordinateType.Absolute, FormatType.xywh,
                                                     img_dim=(width, height))

        for label in labels:
            category_id = int(label[0])
            bbox = list(label[1:5])
            ann_id = len(ann_list)
            ann_list.append(create_annotations_entry(image_id, bbox, category_id, ann_id))
            
    return img_list, ann_list

def create_annotations_dict(target_txt, class_names):
    with open(target_txt, 'r') as f:
        img_path_list = [lines.strip() for lines in f.readlines()]
    label_path_list = [img_path.replace('jpg', 'txt').replace('images', 'labels') for img_path in img_path_list]
    
    img_list, ann_list = get_img_ann_list(img_path_list, label_path_list)
    cat_list = create_categories(class_names)
    
    ann_dict = create_annotations(cat_list, img_list, ann_list)
    return ann_dict

def generate_annotations_file(target_txt, class_names, out):
    ann_dict = create_annotations_dict(target_txt, class_names)
    with open(out, 'w') as f:
        json.dump(ann_dict, f, indent=4, separators=(',', ':'))

In [23]:
generate_annotations_file(valid_target_txt, classes, 'coco_valid.json')

100%|██████████| 5000/5000 [02:01<00:00, 41.32it/s]


### Generate Prediction File

#### COCOEvalDataset

In [24]:
class COCOEvalDataset(Dataset):
    def __init__(self, targ_txt, dim=None, transform=None):
        with open(targ_txt, 'r') as f:
            self.img_list = [lines.strip() for lines in f.readlines()]
        self.label_list = [img_path.replace('jpg', 'txt').replace('images', 'labels') for img_path in self.img_list]
        self.transform = transform
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        label = None
        img_path = self.img_list[idx]
        if osp.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        else:
            raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), img_path)
        
        label_path = self.label_list[idx]
        if osp.exists(label_path):
            label = np.loadtxt(label_path).reshape(-1,5)
            
        sample = { 'img': img, 'org_img': img.copy(), 'label': label, 'transform': None, 'img_path': img_path }
        sample = self.transform(sample)
        
        return sample

#### JsonPredictionWriter BatchHandler

In [25]:
@contextmanager
def open_json_pred_writer(out_path, classes_names, is_letterbox=False):
    pred_writer = JsonPredictionWriter(out_path, classes_names, is_letterbox)
    try:
        pred_writer.write_start()
        yield pred_writer
    finally:
        pred_writer.write_end()
        
class BatchHandler:
    def process_batch(self, sample, predictions):
        raise NotImplementedError
        
class JsonPredictionWriter(BatchHandler):
    def __init__(self, out_path, classes_names, is_letterbox=False):
        self.out_path = out_path
        self.file = open(out_path, 'w')
        self.classes_names = classes_names
        self.is_letterbox = is_letterbox
        
    def write_start(self):
        self.file.write('[')
        
    def write_end(self):
        self.file.seek(self.file.tell() - 1, os.SEEK_SET)
        self.file.truncate()
        self.file.write(']')
        self.file.close()
            
    def process_batch(self, sample, predictions):
        imgs, org_imgs, img_paths = sample['img'], sample['org_img'], sample['img_path']
        for img, org_img, img_path, prediction in zip(imgs, org_imgs, img_paths, predictions):
            img_w, img_h, org_w, org_h = img.shape[2], img.shape[1], org_img.shape[2], org_img.shape[1]
            image_id = get_image_id_from_path(img_path)
            
            if prediction is not None and len(prediction) != 0:
                bboxes = correct_yolo_boxes(prediction[..., 0:4], org_w, org_h, img_w, img_h, self.is_letterbox)
                category_ids = prediction[..., 6]
                scores = prediction[..., 5]
                               
                for category_id, bbox, score in zip(category_ids, bboxes, scores):
                    category_id, bbox, score = int(category_id.item()), bbox.tolist(), score.item()
                    res = create_results_entry(image_id, category_id, bbox, score)
                    json.dump(res, self.file, indent=4, separators=(',', ':'))
                    self.file.write(',')

#### Correct Yolo Boxes (Reverse letterbox or rescale from network image)

In [26]:
def rescale_bbox(labels, org_w, org_h, new_w, new_h):
    if len(labels) == 0:
        return labels

    if isinstance(labels, torch.Tensor):
        labels = labels.clone()
    elif isinstance(labels, np.ndarray):
        labels = labels.copy()
    else:
        raise TypeError("Labels must be a numpy array or pytorch tensor")

    ratio_x, ratio_y = new_w / org_w, new_h / org_h
    mask = labels.sum(-1) != 0
    labels[mask, 0] = np.clip((labels[mask, 0]) / ratio_x, 0, org_w)
    labels[mask, 2] = np.clip((labels[mask, 2]) / ratio_x, 0, org_w)
    labels[mask, 1] = np.clip((labels[mask, 1]) / ratio_y, 0, org_h)
    labels[mask, 3] = np.clip((labels[mask, 3]) / ratio_y, 0, org_h)
    
    return labels

def correct_yolo_boxes(bboxes, org_w, org_h, img_w, img_h, is_letterbox=False):
    if is_letterbox:
        bboxes = letterbox_reverse(bboxes, org_w, org_h, img_w, img_h)
    else:
        bboxes = rescale_bbox(bboxes, org_w, org_h, img_w, img_h)

    bboxes = BoundingBoxConverter.convert(bboxes, 
                                          CoordinateType.Absolute, FormatType.x1y1x2y2,
                                          CoordinateType.Absolute, FormatType.xywh,
                                          img_dim=(img_w, img_h))
    return bboxes

#### Predict and process

In [27]:
def predict_and_process(data, net, num_classes, batch_handler=None):
    with torch.no_grad(): 
        for sample in tqdm(data, file=sys.stdout, leave=True):           
            # Pass images to the network
            det1, det2, det3 = net(sample['img'].cuda(), None)
            predictions = postprocessing(torch.cat((det1,det2,det3), 1),
                                         num_classes, obj_conf_thr=0.005, nms_thr=0.45,
                                         is_eval=True, use_nms=True)
            # Batch Handler - write file
            batch_handler.process_batch(sample, predictions)

In [28]:
def generate_results_file(net, target_txt, classes_names, out, bs, dim, is_letterbox=False):
    numclass = len(classes_names)
    if is_letterbox:
        transform = Compose([IaaAugmentations([IaaLetterbox(dim)]), ToTensor()])
    else:
        transform = Compose([IaaAugmentations([iaa.Scale(dim)]), ToTensor()])

    ds = COCOEvalDataset(target_txt, dim, transform)
    dl = DataLoader(ds, batch_size=bs, num_workers=4, collate_fn=variable_shape_collate_fn)

    with open_json_pred_writer(out, classes_names, is_letterbox) as pred_writer:
        predict_and_process(dl, net, num_classes=numclass, batch_handler=pred_writer)

### Generate Results File

In [29]:
bs = 8
sz = 416
dim = (sz, sz)

In [30]:
net = YoloNet(dim, numClass=80).cuda().eval()
net.loadWeight(weight_path, 'darknet')

generate_results_file(net, valid_target_txt, classes, 'coco_valid_result.json', bs, dim)

100%|██████████| 625/625 [02:30<00:00,  4.16it/s]


### Run pycocotools

In [31]:
cocoGt = COCO("coco_valid.json")

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


In [32]:
resFile = './coco_valid_result.json'
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...
DONE (t=1.81s)
creating index...
index created!


In [33]:
imgIds=sorted(cocoDt.getImgIds())
len(imgIds)

5000

In [34]:
cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=50.51s).
Accumulating evaluation results...
DONE (t=5.86s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.306
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.547
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.309
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.289
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.414
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.272
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.416
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.437
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.230
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10

# CVATDataset (TODO)

# Text File Output (TODO)